In [ ]:
#https://overpass-turbo.eu
import overpy

api = overpy.Overpass()

In [ ]:
result = api.query("""
    [out:json];
    nwr["railway"="rail"](44.80219,11.660423,44.815496,11.695951);
    out geom;
    """)

print(f"{result}")


In [56]:
from storage.storage_engine import FileStorage

file_storage = FileStorage()
s, df_h3_info = file_storage.load_dataframe('L', file_storage.h3_info, '2025')

In [57]:
import h3

target_resolution = 8

# Crea una nuova colonna con l'H3 parent
df_h3_info['h3_parent'] = df_h3_info['h3'].apply(lambda x: h3.cell_to_parent(x, target_resolution))

# Tieni solo una riga per ogni coppia (track_id, h3)
df_unique = df_h3_info.drop_duplicates(subset=['track_id', 'h3_parent'])

# Conta quante tracce distinte hanno attraversato ogni h3
h3_counts = df_unique['h3_parent'].value_counts().reset_index()
h3_counts.columns = ['h3_parent', 'distinct_track_count']

h3_counts_filtered = h3_counts[h3_counts['distinct_track_count'] >= 15]

#print(h3_counts)

In [ ]:
# aggiungo i colori
import matplotlib.pyplot as plt
from matplotlib import colors

# Scegli una colormap, ad esempio 'viridis'
cmap = plt.get_cmap('viridis')

# Normalizza i valori tra 0 e 1
norm = plt.Normalize(h3_counts_filtered['distinct_track_count'].min(), h3_counts_filtered['distinct_track_count'].max())

# Applica la colormap e converti in esadecimale
h3_counts_filtered['color'] = h3_counts_filtered['distinct_track_count'].apply(lambda x: colors.to_hex(cmap(norm(x))))

# Ora h3_counts ha una colonna 'color' con il colore associato

In [60]:
from shapely.geometry import shape, Polygon
import geopandas as gpd

def h3_to_geojson(h3_cell):
    """Convert H3 index to GeoJSON Polygon."""
    boundary = h3.cells_to_geo([h3_cell])
    return Polygon(shape(boundary))

h3_geoms = h3_counts_filtered["h3_parent"].apply(lambda x: h3_to_geojson(x))
h3_gdf = gpd.GeoDataFrame(data=h3_counts_filtered, geometry=h3_geoms, crs=4326)


In [61]:
h3_gdf.to_file("files/L/2025_h3_counts.geojson", driver='GeoJSON')

In [ ]:
h3_gdf.info()

In [62]:
import folium

# Centra la mappa su una coordinata a tua scelta
m = folium.Map(location=[45.8566, 9.3977], zoom_start=10)

# Aggiungi i poligoni colorati
for _, row in h3_gdf.iterrows():
    geojson = folium.GeoJson(row['geometry'].__geo_interface__,
                             style_function=lambda feature, color=row['color']: {
                                 'fillColor': color,
                                 'color': color,
                                 'weight': 1,
                                 'fillOpacity': 0.6
                             })
    geojson.add_to(m)

m

In [2]:
import pandas as pd
df = pd.read_parquet("files/L/2025_tracks_info.parquet", engine="pyarrow")
df.to_csv("files/L/2025_tracks_info.csv", index=False)